In [2]:
import ipywidgets as widgets
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython.display import display
import geopandas as gpd
import os
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import plotly.express as px  # For interactive tooltips



# Creates map with 26 counties
os.environ['SHAPE_RESTORE_SHX'] = 'YES'
shapefile_path = os.path.join("./26 Counties/", "Census2011_Admin_Counties_generalised20m.shp")

if not os.path.exists(shapefile_path):
    print(f"Error: Shapefile not found at {shapefile_path}")
    exit()

try:
    gdf = gpd.read_file(shapefile_path)
except Exception as e:
    print(f"Error reading shapefile: {e}")
    exit()


gdf["COUNTYNAME"] = gdf["COUNTYNAME"].str.replace(r" County$", "", regex=True)
county_ratings = {
    "Carlow": 3,
    "Cavan": 6,
    "Clare": 8,
    "Cork": 9,
    "Donegal": 5,
    "Dublin": 10,
    "Galway": 7,
    "Kerry": 8,
    "Kildare": 6,
    "Kilkenny": 7,
    "Laois": 4,
    "Leitrim": 5,
    "Limerick": 8,
    "Longford": 3,
    "Louth": 6,
    "Mayo": 7,
    "Meath": 7,
    "Monaghan": 4,
    "Offaly": 5,
    "Roscommon": 6,
    "Sligo": 6,
    "Tipperary": 7,
    "Waterford": 8,
    "Westmeath": 5,
    "Wexford": 6,
    "Wicklow": 9,
}

gdf['rating'] = gdf['COUNTYNAME'].map(county_ratings)
gdf['rating'] = gdf['rating'].fillna(0)
gdf = gdf.drop(columns=["Male2011",'Female2011','TOTAL_AREA','LAND_AREA','PPOcc2011', 'Unocc2011', 'HS2011', 'Vacant2011', 'PCVac20111'], errors='ignore')



Data Set on waiting time for test at end of month

In [3]:
#https://data.gov.ie/dataset/roa37-driving-test-waiting-time/resource/03c147f2-dc46-4405-814e-8a7ad43d145d?inner_span=True



csv_file_path = "waitingTimes.csv" 

try:
    df = pd.read_csv(csv_file_path)
except Exception as e:
    print(f"Error reading CSV file: {e}")
    exit()



df.rename(columns={"County": "COUNTYNAME"}, inplace=True)
df["COUNTYNAME"] = df["COUNTYNAME"].str.strip()
df["COUNTYNAME"] = df["COUNTYNAME"].str.replace(r"^Co\. ", "", regex=True)
df.rename(columns={"VALUE": "WaitingTime"}, inplace=True)


# Merging with gdf
merged_gdf = gdf.merge(df[['COUNTYNAME', 'TLIST(M1)', 'WaitingTime']], on='COUNTYNAME', how='left')



Data Set on number of driving test applications received, delivered, pass rate... and then also finds number of test centres by county

In [6]:
#https://www.cso.ie/en/releasesandpublications/hubs/p-transo/transporthub/driverandvehicletesting/drivingteststatistics/
csv_file_path = "driving_tests.csv" 
try:
    df = pd.read_csv(csv_file_path)
except Exception as e:
    print(f"Error reading CSV file: {e}")
    exit()


def convert_month_to_tlist(month_string):
    
    try:
        year, month_name = month_string.split()  
        year = int(year)

        month_dict = {
            'january': 1, 'february': 2, 'march': 3, 'april': 4,
            'may': 5, 'june': 6, 'july': 7, 'august': 8,
            'september': 9, 'october': 10, 'november': 11, 'december': 12
        }

        month = month_dict[month_name.lower()]  
        tlist = year * 100 + month  
        return tlist
    except (ValueError, KeyError): 
        return None


df['TLIST(M1)'] = df['Month'].apply(convert_month_to_tlist)



def clean_driving_centre(centre_name):
    """Cleans the driving centre name, keeping the last word if 'Co.' is present,
       otherwise returning 'Ireland'.
    """
    if isinstance(centre_name, str):  
        if "Co." in centre_name:
            words = centre_name.split()
            try:
                co_index = words.index("Co.") 
            except ValueError:
                try:
                    co_index = words.index("County") 
                except ValueError:
                    return "Ireland" 

            if co_index + 1 < len(words): 
                return words[co_index + 1]
            else:
                return "Ireland"
        else:
            return "Ireland"
    else:
        return "Ireland" 


df['Cleaned Driving Centres'] = df['Driving Test Centre'].apply(clean_driving_centre)
df = df.drop(columns=['UNIT'])
county_counts = df['Cleaned Driving Centres'].value_counts().reset_index()
county_counts.columns = ['COUNTYNAME', 'Number of Test Centres']
county_counts['Number of Test Centres'] =county_counts['Number of Test Centres']/4320


# Merging with gdf
merged_gdf = merged_gdf.merge(county_counts[['COUNTYNAME', 'Number of Test Centres']], on='COUNTYNAME', how='left')


Data on applications...

In [7]:
#https://www.cso.ie/en/releasesandpublications/hubs/p-transo/transporthub/driverandvehicletesting/drivingteststatistics/
csv_file_path = "driving_tests.csv" 
try:
    df = pd.read_csv(csv_file_path)
except Exception as e:
    print(f"Error reading CSV file: {e}")
    exit()


def convert_month_to_tlist(month_string):
    
    try:
        year, month_name = month_string.split()  
        year = int(year)
        month_dict = {
            'january': 1, 'february': 2, 'march': 3, 'april': 4,
            'may': 5, 'june': 6, 'july': 7, 'august': 8,
            'september': 9, 'october': 10, 'november': 11, 'december': 12
        }

        month = month_dict[month_name.lower()]  

        tlist = year * 100 + month 
        return tlist
    except (ValueError, KeyError):  
        return None


df['TLIST(M1)'] = df['Month'].apply(convert_month_to_tlist)
unique_centres = df["Driving Test Centre"].unique()
df["County"] = df["Driving Test Centre"].str.extract(r', Co\. (\w+)$')


df_county = df.groupby(["County", "Statistic Label", "TLIST(M1)", "Driving Test Categories"])["VALUE"].sum().reset_index()
df_category_b = df_county[df_county["Driving Test Categories"] == "Category B (Car or light van)"]


df_pivot = df_category_b.pivot_table(
    index=["County", "TLIST(M1)"], 
    columns="Statistic Label",  
    values="VALUE", 
    aggfunc="sum"
).reset_index()

df_pivot.rename(columns={"County": "COUNTYNAME"}, inplace=True)

merged_gdf = merged_gdf.merge(df_pivot, on=['COUNTYNAME','TLIST(M1)'], how='inner')


This section is using the wrong "NCT" data set lol 

In [8]:
#https://www.rsa.ie/road-safety/statistics/nct-statistics-and-annual-reviews
csv_file_path = "2023-Pass-Fail-Rates-by-Test-Centre(2023)_new.csv"  # Update with your actual CSV file path
try:
    df = pd.read_csv(csv_file_path)
except Exception as e:
    print(f"Error reading CSV file: {e}")
    exit()



df = df.iloc[:-1]
df.columns = df.columns.str.strip()

for col in df.columns:
    if col != "COUNTYNAME":  
        df[col] = pd.to_numeric(df[col], errors="coerce")

df_grouped = df.groupby("COUNTYNAME").sum()
df_grouped=df_grouped.drop(columns=["Centre"], errors="ignore")
df_grouped['Pass Rates']=(df_grouped['PASS (Initial Test)']+df_grouped['PASS (Re-test)'])/(df_grouped['PASS (Initial Test)']+df_grouped['PASS (Re-test)']+df_grouped['FAIL (Initial Test)']+df_grouped['FAIL (Re-test)'])

gdf_merged=merged_gdf = merged_gdf.merge(df_grouped, on=['COUNTYNAME'], how='inner')


Interactive map with hover labels

In [9]:
import ipywidgets as widgets
import plotly.express as px
from IPython.display import display

# Dropdown for Year
year_dropdown = widgets.Dropdown(
    options=[str(year) for year in range(2021, 2025)],  # Restrict years from Sept 2021 - Dec 2024
    description='Year:',
    value='2024'
)

# Dropdown for Month
month_dict = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}
# Dropdown for Exam Length  (1h, 55m, etc.)
time_options = ["1h", "55m", "50m", "45m", "40m"]
time_dropdown = widgets.Dropdown(
    options=time_options,
    description='Proposed Exam Length:',
    value='1h'  # Default value
)
# Percentage slider for pass rate
percentage_slider = widgets.IntSlider(
    value=50,  # Initial value
    min=0,
    max=100,
    step=1,  # Increment by 1
    description='Target Pass Rate:',
    disabled=False,
    continuous_update=False,  # Update only on release (optional, but good for performance)
    orientation='horizontal',
    readout=True,  # Display the current value
    style={'description_width': 'initial'} # Improves label alignment
)



month_dropdown = widgets.Dropdown(
    options=list(month_dict.keys()),
    description='Month:',
    value='January'
)

# Button
select_button = widgets.Button(description="Filter Data")

# Output display
output = widgets.Output()


        
def normalize(series):
    return (series - series.min()) / (series.max() - series.min()) * 10

def calculate_rating(df):
    # Normalize metrics (invert waiting time and waiting applicants since lower is better)
    df["WaitingTime_Score"] = 10 - normalize(df["WaitingTime"])
    df["TestCentres_Score"] = normalize(df["Number of Test Centres"])
    df["NotEligible_Score"] = 10 - normalize(df["Driving Test Applicants Not Eligible at Month End"])
    df["Paused_Score"] = 10 - normalize(df["Driving Test Applicants Paused at Month End"])
    df["Scheduled_Score"] = normalize(df["Driving Test Applicants Scheduled at Month End"])
    df["WaitingApplicants_Score"] = 10 - normalize(df["Driving Test Applicants Waiting at Month End"])
    df["ApplicationsReceived_Score"] = normalize(df["Driving Test Applications Received"])
    df["PassRate_Score"] = normalize(df["Pass Rates"])
    
    # Define weights
    weights = {
        "WaitingTime_Score": 0.2,
        "TestCentres_Score": 0.15,
        "NotEligible_Score": 0.1,
        "Paused_Score": 0.1,
        "Scheduled_Score": 0.15,
        "WaitingApplicants_Score": 0.1,
        "ApplicationsReceived_Score": 0.1,
        "PassRate_Score": 0.1,
    }
    
    # Compute final rating
    df["Rating"] = sum(df[col] * weight for col, weight in weights.items())
    
    # Assign rating categories
    df["Rating Category"] = pd.cut(df["Rating"], bins=[0, 2, 4, 6, 8, 10],
                                     labels=["Poor", "Needs Improvement", "Average", "Good", "Excellent"],
                                     include_lowest=True)
    
    return df


def on_button_click(b):
    with output:
        output.clear_output()
        selected_year = int(year_dropdown.value)
        selected_month = month_dict[month_dropdown.value]
        selected_tlist = selected_year * 100 + selected_month
        selected_time = time_dropdown.value # Get selected time
        selected_percentage= percentage_slider.value/100

        
        # Filter data
        filtered_gdf = merged_gdf[merged_gdf['TLIST(M1)'] == selected_tlist].copy()  # Copy at the earliest stage

        # Apply calculation to 'rating' based on 'selected_time'
        if selected_time == "1h":
            filtered_gdf['WaitingTime'] = filtered_gdf['WaitingTime'] * 1.0  
        elif selected_time == "55m":
            filtered_gdf['WaitingTime'] = filtered_gdf['WaitingTime'] * .95 
        elif selected_time == "50m":
            filtered_gdf['WaitingTime'] = filtered_gdf['WaitingTime'] * 0.9 
        elif selected_time == "45m":
            filtered_gdf['WaitingTime'] = filtered_gdf['WaitingTime'] * 0.85  
        elif selected_time == "40m":
            filtered_gdf['WaitingTime'] = filtered_gdf['WaitingTime'] * 0.8  


        filtered_gdf.loc[filtered_gdf["Pass Rates"] < selected_percentage, "Pass Rates"] = (selected_percentage)
        filtered_gdf = calculate_rating(filtered_gdf)
        
       
        if filtered_gdf.empty:
            print(f"No data found for {month_dropdown.value} {year_dropdown.value}")
        else:
            filtered_gdf = filtered_gdf.to_crs("EPSG:4326")  # Convert back to WGS84 for plotting

            # Reproject for centroid calculation *before* calculating centroid
            projected_crs = "EPSG:3857"  # Example: Web Mercator (choose an appropriate projected CRS)
            filtered_gdf = filtered_gdf.to_crs(projected_crs)

            # Now calculate the centroid
            center_lat = filtered_gdf.geometry.centroid.y.mean()
            center_lon = filtered_gdf.geometry.centroid.x.mean()

            # Reproject back to WGS84 for plotting
            filtered_gdf = filtered_gdf.to_crs("EPSG:4326")
            
            fig = px.choropleth_map( 
                filtered_gdf,
                geojson=filtered_gdf.geometry,
                locations=filtered_gdf.index,
                color="Rating",
                hover_name="COUNTYNAME",
                hover_data=["Rating Category","WaitingTime","Number of Test Centres","Driving Test Applicants Not Eligible at Month End","Driving Test Applicants Paused at Month End","Driving Test Applicants Scheduled at Month End","Driving Test Applicants Waiting at Month End","Driving Test Applications Received","Pass Rates"],
                center={"lat": filtered_gdf.geometry.centroid.y.mean(), "lon": filtered_gdf.geometry.centroid.x.mean()},
                zoom=7,
                title=f"Map of WaitingTime by COUNTYNAME for {month_dropdown.value} {year_dropdown.value}",
                color_continuous_scale='RdYlGn'
            )
            fig.update_geos(fitbounds="locations", visible=False)
            fig.update_layout(margin={"r": 0, "t": 50, "l": 0, "b": 0})
            fig.show()

# Attach event
select_button.on_click(on_button_click)

# Display widgets
display(year_dropdown, month_dropdown,time_dropdown, percentage_slider, select_button, output)


Dropdown(description='Year:', index=3, options=('2021', '2022', '2023', '2024'), value='2024')

Dropdown(description='Month:', options=('January', 'February', 'March', 'April', 'May', 'June', 'July', 'Augus…

Dropdown(description='Proposed Exam Length:', options=('1h', '55m', '50m', '45m', '40m'), value='1h')

IntSlider(value=50, continuous_update=False, description='Target Pass Rate:', style=SliderStyle(description_wi…

Button(description='Filter Data', style=ButtonStyle())

Output()

In [7]:
#print(merged_gdf)
#merged_gdf.to_csv("merged_gdf.csv")